In [1]:
import pandas as pd
import numpy as np
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)  
from sklearn import preprocessing, cross_validation
from sklearn.neural_network import MLPClassifier

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
def make_network(FILENAME):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    from sklearn.neural_network import MLPClassifier
    """Given the csv input of all the box scores, arrange it such that the home and away teams are lined up, 
    unnecessary columns removed, and proper hot encoding is done. Other stuff too probably.
    
    Note that this data has already been doctored from its original form, taking out most unnecessary columns but
    those could be useful later on.
    
    
    Parameters
    ----------
    FILENAME : file
        The csv of the data.
        
    Returns
    -------
    
    X_train : array
        Numpy array of the training inputs.
    y_train : array
        Numpy array of the training outputs.
    
    X_test : array
        Numpy array of the testing inputs.
    y_test : array
        Numpy array of the testing outputs.
    Sike!
    
    model : object
        MLP which can predict the outcome of NBA games
    """
    #Read in file, remove attempted and # and only account for % since that's more predictive in nature. 
    data = read_csv(FILENAME)
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values)
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['FT'],data['FTA']
    data = get_dummies(data)
    del data['VENUE_Home'],data['VENUE_Road']
    #print(data)
    
    dat = []

    for i in range(len(data.values)):
        data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
    for p in range(int(len(data.values)/2)):
        fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
        dat.append(fullboxgame)
            
    """At this point in the data dat is an array arranged as 
        OR  DR  TOT   A  PF  ST  TO  BL  PTS       3P%       FG%       FT% x2  (for road then home )
    so next up is to hot pull road and home points 
    
    road points is column 8 and home points is column 20. 
    
    So grab those from dat and make y. 
    
    X is every other column, so merge everyone else 
    
    """
    dat = np.array(dat)    
    y = []
    for j in range(len(dat)):
        roadpts = dat[j,8]
        homepts = dat[j,20]
       # print(home,road)
    #distinguish home win or road win. 
        if homepts > roadpts:
            y.append(np.array([0,1]))  # 0 1 mean home won
        else:
            y.append(np.array([1,0]))  #  1 0 means away won. 
    y = np.array(y)
    X1 = np.concatenate((dat[:,0:8],dat[:,9:20]),axis=1)
    X = np.concatenate((X1,dat[:,21:24]),axis=1)    #need to go one further column to snag HFT% 
    X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.4)
    model = MLPClassifier()
    model.batch_size = 20
    model.n_layers_ = 5
    model.n_outputs_= 30
    model.fit(X_train,y_train)
    print(model.score(X_test,y_test))
    return model
    

In [7]:
noahbets = make_network('1517_boxscores.csv')

0.8352380952380952


In [8]:
def make_prediction_data(filename):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    data = read_csv(filename)
    #convert to per game stats and sort columns. 
    data['ORB'] =  np.divide(data['ORB'].values,data['G'].values)
    data['DRB'] =  np.divide(data['DRB'].values,data['G'].values)
    data['TRB'] =  np.divide(data['TRB'].values,data['G'].values)
    data['AST'] =  np.divide(data['AST'].values,data['G'].values)
    data['STL'] =  np.divide(data['STL'].values,data['G'].values)
    data['BLK'] =  np.divide(data['BLK'].values,data['G'].values)
    data['TOV'] =  np.divide(data['TOV'].values,data['G'].values)
    data['PF'] =  np.divide(data['PF'].values,data['G'].values)
    teams  = data['Team']

    data = data[['ORB' , 'DRB'  ,'TRB' ,  'AST' , 'PF' , 'STL' , 'TOV' , 'BLK' ,'3P%','FG%' ,'FT%']]
    print("Here is every teams index value: ")
    print(teams)
    return teams,data

In [9]:
def game_maker(roadteam,hometeam):
    """After creating a properly formated table, this concats the desired teams so they can be predicted. 
        Based on get team index # based on output of predictor, and make it the input for stats ie GSW are stats[0].
        and so on!
    """
    game = np.concatenate((roadteam,hometeam))
    return game

In [10]:
teams, stats =make_prediction_data('3-12teamstats.csv')

Here is every teams index value: 
0      Golden State Warriors
1            Houston Rockets
2     Minnesota Timberwolves
3           Toronto Raptors*
4       New Orleans Pelicans
5             Denver Nuggets
6        Cleveland Cavaliers
7      Oklahoma City Thunder
8         Washington Wizards
9         Los Angeles Lakers
10         Charlotte Hornets
11      Los Angeles Clippers
12              Phoenix Suns
13            Indiana Pacers
14             Brooklyn Nets
15             Orlando Magic
16        Philadelphia 76ers
17            Boston Celtics
18    Portland Trail Blazers
19           New York Knicks
20             Atlanta Hawks
21           Milwaukee Bucks
22                 Utah Jazz
23                Miami Heat
24          Dallas Mavericks
25             Chicago Bulls
26           Detroit Pistons
27         San Antonio Spurs
28          Sacramento Kings
29         Memphis Grizzlies
30            League Average
Name: Team, dtype: object


In [12]:
game1 = game_maker(stats.values[13],stats.values[16])
game2 = game_maker(stats.values[2],stats.values[8])
game3 = game_maker(stats.values[7],stats.values[20])
game4 =  game_maker(stats.values[3],stats.values[14])
game5 = game_maker(stats.values[24],stats.values[19])
game6 = game_maker(stats.values[11],stats.values[25])
game7 = game_maker(stats.values[10],stats.values[4])
game8 = game_maker(stats.values[15],stats.values[27])
game9 = game_maker(stats.values[26],stats.values[22])
game10 =game_maker(stats.values[6],stats.values[12])
game11 =game_maker(stats.values[5],stats.values[9])

In [13]:
noahbets.predict(np.array([game1,game2,game3,game4,game5,game6,game7,game8,game9,game10,game11]))

array([[0, 1],
       [0, 1],
       [1, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1]])

Winners 3/10:
10 Hornets Y
24 Heat Y
27 Spurs N
26 Mavs Y
12 Clippers Y

Winners 3/11:
4 Raptors Y
25 Bulls Y
0 GSW N
3 Pelicans N
5 Nuggets Y
26 Rockets Y
13 Nets N
7 Cavs N
18 Celtics N

Winners 3/12:
Blazers Y
Spurs N
Bucks Y
Thunder Y


Winners 3/13:

Philly N
Wash N
OKC Y
Toronto Y
Dallas Y
Bulls N
Pels Y
Spurs Y
UTAH Y
Cle Y
Lakers Y

20-9

Predicted winners on March 9th:

Indiana
Chicago 
toronto
nop
utah
knicks    *
lakers
orlando  *
warriors
clippers


In [2]:
20/29

0.6896551724137931

Arrangement of Data:
Away :      OR  DR  TOT   A  PF  ST  TO  BL  PTS   3P%       FG%       FT%
  Home: OR  DR  TOT   A  PF  ST  TO  BL  3P%  FG%  FT%| AW HW
      0    1   2   3   4   5  6    7   8    9   10  

Data:
1. Road Orbs
2. Road Drbs
3. Road Trbs
4. Road Asts
5. Road Pfs
6. Road Stls
7. Road Tos. 
8. Road Blks 
9. Road 3p%
10. Road Fg%
11. Road Ft% 
12. Road venue Home (0)  REMOVED
13. Road venue Away (1) REMOVED
14. Home Orbs
15. Home Drbs
16. Home Trbs
17. Home Asts
18. Home Pfs
19. Home Stls
20. Home Tos. 
21. Home Blks 
22. Home 3p%
23. Home Fg%
24. Home Ft% 
25. Home venue Home (0) REMOVED
26. Home venue Away (1) REMOVED

With Outputs 

27. Home team win (1)
28. Road team win (0)


So you enter the statistics of both teams, and it gives a classification of whether or not the home or away team wins.
Unclear how good this actually is at characterizing teams. Should use team stats from as recent a metric as possible to reflect current gameplay


Let's do a raw version where I pull the last 5 stats of wiz vs pels 

3/8 Games and if it was correct:

Suns @ OKC YES

Nets @ HOR NO

Bos @ Min NO

Phi @ Mia YES

Sas @ GSW Says spurs, 

This is a good start. What's left?

1. What about assists, rebounds, steals, etc.? Can a table do that too? 

2. Hot encode wins and losses. Need to move that result column and delete everyhting except W and L.

3. Normalize data?

4. Create a net

5. Figure out prediction data, go by season averages? L5 averages? 

Remove unneccesary stats. Flatten 2 adjascent rows because they're the same game!